In [25]:
import ee
ee.Initialize(project='gtac-lamda')
import geeViz.getImagesLib as gil
import geeViz.geePalettes as palette
import geeViz.assetManagerLib as aml
import geeViz.changeDetectionLib as cdl
import math,os

Map = gil.Map

In [7]:

Map.clearMap()
Map.port = 1234
# Set projection (for visualizing, must reproject terrain-based outputs)
output_collection = 'projects/lcms-292214/assets/Ancillary/Wetland/Height_Above_Channel'
aml.create_asset(output_collection,ee.data.ASSET_TYPE_IMAGE_COLL)


# Get NHD flowlines and waterbodies
# Filter out as needed: https://www.usgs.gov/ngp-standards-and-specifications/national-hydrography-dataset-nhd-data-dictionary-feature-classes
nhd_files  = [f['id'] for f in ee.data.getList({'id':"projects/sat-io/open-datasets/NHD"})]

# flowlines = ee.FeatureCollection([ee.FeatureCollection(f+'/NHDFlowline') for f in nhd_files]).flatten()
# waterbodies = ee.FeatureCollection([ee.FeatureCollection(f+'/NHDWaterbody') for f in nhd_files]).flatten()
states = ee.FeatureCollection("TIGER/2018/States")

nwi_obj_info = {
      'Wetland_Class_class_names': [
        "Freshwater Forested/Shrub Wetland",
        "Freshwater Emergent Wetland",
        "Freshwater Pond",
        "Estuarine and Marine Wetland",
        "Riverine",
        "Lake",
        "Estuarine and Marine Deepwater",
        "Other",
      ],
      'Wetland_Class_class_palette': [
        "008837",
        "7FC31C",
        "688CC0",
        "66C2A5",
        "0190BF",
        "13007C",
        "007C88",
        "B28653",
      ],
      'Wetland_Class_class_values': [1, 2, 3, 4, 5, 6, 7, 8],
      'bandNames': ["Wetland_Class"],
    }
nwi = ee.ImageCollection('projects/lcms-292214/assets/Ancillary/NWI').mosaic().set(nwi_obj_info)
Map.addLayer(nwi,{'autoViz':True},'NWI')
def getHAC(flowlines,waterbodies,nm,crs,transform):
    flowlines = flowlines.filter(ee.Filter.eq('ftype',460))
    
    waterbodies = waterbodies.filter(ee.Filter.eq('ftype',390))

    # Merge NHD and set a dummy property for rasterizing
    nhd= flowlines.merge(waterbodies)
    nhd = nhd.map(lambda f:f.set('prop',1))
    Map.addLayer(nhd,{'styleParams':{'lineType':'dashed','color':'80F','fillColor':'0FF3'}},f'{nm}- NHD Flowlines and Waterbodies',False)


    # Rasterize NHD
    nhd_rast = nhd.reduceToImage(['prop'],ee.Reducer.first())
    nhd_rast = nhd_rast.mask().Or(nwi.mask()).selfMask()
    
    Map.addLayer(nhd_rast,{'min':1,'max':1,'palette':'00D'},f'{nm}- NHD Rast',False)

    # Get distance raster (just for visualizing)
    dist = nhd_rast.distance(ee.Kernel.euclidean(1000,'meters'),False)
    Map.addLayer(dist.reproject(crs,transform),{'min':0,'max':1000,'palette':palette.matplotlib['magma'][7]},f'{nm}-Distance From NHD (meters)',False)

    # Bring in terrain data
    dem = ee.Image("USGS/3DEP/10m").resample('bicubic')#.focalMean(3.5,'circle','pixels')


    # Compute slope in radians
    slope =ee.Terrain.slope(dem).multiply(math.pi).divide(180)
    Map.addLayer(slope.reproject(crs,transform),{'min':0,'max':0.4,'palette':palette.matplotlib['inferno'][7]},f'{nm}-Slope (Radians)',False)


    # Compute the height above NHD 
    hac = slope.tan().cumulativeCost(nhd_rast,1500).rename(['Meters_Above_Channel']).int16()
    Map.addLayer(hac.reproject(crs,transform),{'min':0,'max':80,'palette':palette.matplotlib['plasma'][7]},f'{nm}-Height Above Channel (meters)',True)

    # Mask for valley bottom
    vb = hac.lte(3).selfMask().rename('Valley_Bottom').set({'Valley_Bottom_class_names':['Valley Bottom'],'Valley_Bottom_class_values':[1],'Valley_Bottom_class_palette':['0088FF']})
    Map.addLayer(vb.reproject(crs,transform),{'autoViz':True}, f'{nm}-Valley Bottom',True)
    return hac

# nhd_files = [f for f in nhd_files if f.find('NHD_ID')>-1]
for nhd_file in nhd_files:
    
    # print(nhd_file)
    nm = os.path.basename(nhd_file).split('NHD_')[1]
    state = states.filter(ee.Filter.eq('STUSPS',nm)).first()
    print(nm,state.get('STUSPS').getInfo())
    crs = gil.common_projections['NLCD_CONUS']['crs']
    transform = gil.common_projections['NLCD_CONUS']['transform']

    if os.path.basename(nhd_file).find('_AK')> -1:
        crs = gil.common_projections['NLCD_AK']['crs']
        transform = gil.common_projections['NLCD_AK']['transform']
    elif os.path.basename(nhd_file).find('_HI')> -1:
        crs = gil.common_projections['NLCD_HI']['crs']
        transform = gil.common_projections['NLCD_HI']['transform']
    res = 10
    transform[0] = res
    transform[4] = -res

    flowlines = ee.FeatureCollection(nhd_file+'/NHDFlowline')
    waterbodies = ee.FeatureCollection(nhd_file+'/NHDWaterbody') 
    hac = getHAC(flowlines,waterbodies,nm,crs,transform)

    output_name = f'{nm}_Height_Above_NHD_Flowline_and_Waterbody_{res}m'
    output_asset = f'{output_collection}/{output_name}'
    gil.exportToAssetWrapper(hac,output_name,output_asset,roi =state,crs = crs,transform = transform,overwrite=False)

hac = ee.ImageCollection(output_collection).mosaic()

Map.addLayer(hac,{'min':0,'max':80,'palette':palette.matplotlib['plasma'][7]},f'Asset Height Above Channel (meters)',True)

Map.turnOnInspector()
Map.setQueryTransform(transform)
# Map.view(open_browser=True)


Found the following sub directories:  ['Ancillary', 'Wetland', 'Height_Above_Channel']
Will attempt to create them if they do not exist
Asset projects/lcms-292214/assets/Ancillary already exists
Asset projects/lcms-292214/assets/Ancillary/Wetland already exists
Asset projects/lcms-292214/assets/Ancillary/Wetland/Height_Above_Channel already exists
Adding layer: NWI
AK AK
Adding layer: AK- NHD Flowlines and Waterbodies
Adding layer: AK- NHD Rast
Adding layer: AK-Distance From NHD (meters)
Adding layer: AK-Slope (Radians)
Adding layer: AK-Height Above Channel (meters)
Adding layer: AK-Valley Bottom
AK_Height_Above_NHD_Flowline_and_Waterbody_10m currently exists or is being exported and overwrite = False. Set overwite = True if you would like to overwite any existing asset or asset exporting task
AL AL
Adding layer: AL- NHD Flowlines and Waterbodies
Adding layer: AL- NHD Rast
Adding layer: AL-Distance From NHD (meters)
Adding layer: AL-Slope (Radians)
Adding layer: AL-Height Above Channel

In [34]:
hac = ee.ImageCollection(output_collection).mosaic()
Map.clearMap()

dem = ee.Image("USGS/3DEP/10m")
angles = [270]#[90,180,270]
triShade = ee.Image.cat([ee.Terrain.hillshade(dem,a) for a in angles])
Map.addLayer(triShade,{'min':0,'max':255},'Hillshade')
Map.addLayer(hac,{'opacity':0.3,'min':0,'max':30,'palette':palette.matplotlib['plasma'][7]},f'Height Above Channel (meters)',True)


vb = hac.lte(1).selfMask().rename('Valley_Bottom').set({'Valley_Bottom_class_names':['Valley Bottom'],'Valley_Bottom_class_values':[1],'Valley_Bottom_class_palette':['0055DD']})
Map.addLayer(vb,{'autoViz':True}, f'Valley Bottom',True)
Map.addLayer(nwi,{'autoViz':True},'NWI',False)


Map.turnOnInspector()
Map.view(open_browser=True)


Adding layer: Hillshade
Adding layer: Height Above Channel (meters)
Adding layer: Valley Bottom
Adding layer: NWI
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:1234/geeView/ already serving.
cwd z:\Projects\06_LCMS_4_NFS\Scripts\landscape-change-data-explorer\src\setup
geeView URL: http://localhost:1234/geeView/?projectID=gtac-lamda&accessToken=ya29.a0AeDClZAl6cIeWZPoxuw0A1jMDEzIKd4UPNIlgmv0pGapBZgyrfUoYunfEGnfXK4ysgNKooLwRXcKx-YwLZdBYZJRkefEnEnl3NQAqII7L6fQz6082PHH3QKX07DOuclOddPT3pwLSziogMTofgasAlKr-q7W5V-XBSIQGdvrSnEaCgYKAdkSARESFQHGX2MiVzf9sTeTt_q4NMkyczLpRw0178


In [17]:
Map.clearMap()

sa_dict = {'CONUS':{'studyArea' :ee.FeatureCollection('projects/lcms-292214/assets/CONUS-Ancillary-Data/conus')}}
startJulian = 274
endJulian = 273
startYear = 1984
endYear = 2023
timebuffer = 0
weights = [1]
compositingReducer = ee.Reducer.mean()
collectionName = "NASA/ORNL/DAYMET_V4"
exportComposites = True
exportPathRoot =  'projects/lcms-292214/assets/Ancillary/Wetland/Annual_Climate'
aml.create_asset(exportPathRoot,ee.data.ASSET_TYPE_IMAGE_COLL)
scale = 990
# exportBands = ["prcp.*", "tmax.*", "tmin.*","swe.*"]
exportBands = [ 'prcp.*', 'srad.*', 'swe.*', 'tmax.*', 'tmin.*', 'vp.*']

sa = 'CONUS'
crs = gil.common_projections['NLCD_'+sa]['crs']
transform =gil.common_projections['NLCD_'+sa]['transform']
transform[0] = scale
transform[4] = -scale
print(crs)
print(transform)
studyArea = sa_dict[sa]['studyArea'].geometry().bounds(500,crs).buffer(20000)
scale = None
climateComposites = gil.getClimateWrapper(collectionName, studyArea, startYear, endYear, startJulian, endJulian, timebuffer, weights, compositingReducer,exportComposites, exportPathRoot, crs, transform, scale, exportBands)
Map.addLayer(climateComposites.select(exportBands), {}, "Climate Composite Time Lapse")
Map.addLayer(studyArea, {"strokeColor": "0000FF", "canQuery": False}, "Study Area", True)
Map.centerObject(studyArea)
Map.turnOnInspector()
Map.view()
# print(ee.ImageCollection("NASA/ORNL/DAYMET_V4").first().bandNames().getInfo())

Found the following sub directories:  ['Ancillary', 'Wetland', 'Annual_Climate']
Will attempt to create them if they do not exist
Asset projects/lcms-292214/assets/Ancillary already exists
Asset projects/lcms-292214/assets/Ancillary/Wetland already exists
Asset projects/lcms-292214/assets/Ancillary/Wetland/Annual_Climate already exists
PROJCS["Albers_Conical_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],TOWGS84[0,0,0,0,0,0,0],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
[990, 0, -2361915.0, 0, -990, 3177735.0]

In [20]:
Map.clearMap()
wells = ee.FeatureCollection("projects/sat-io/open-datasets/USGWD-TABULAR-MERGED")
Map.addLayer(wells,{'styleParams':{'pointSize':5}},'Wells')

gde = ee.ImageCollection("projects/sat-io/open-datasets/GlobalGDEMap_v6_TNC").mosaic()

Map.addLayer(gde.select([0]).selfMask(),{'palette':'00F'},'GDE')

Map.turnOnInspector()
Map.view(open_browser=True)


Adding layer: Wells
Adding layer: GDE
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:8001/geeView/ already serving.
cwd z:\Projects\06_LCMS_4_NFS\Scripts\landscape-change-data-explorer\src\setup
geeView URL: http://localhost:8001/geeView/?projectID=gtac-lamda&accessToken=ya29.a0AeDClZAsxB_uCOY3V9Z5YdDWnjcRdFCWP4Xy_2fPYqbfa_kDA6-JX6VciY9uRTMReQc3pRBucrDnwTGbADWNCSwxqY8aClFMZa8xlbYtGk8xMtFIX1Z1jy74Va-0-jvdTNaPkPZ1cOPWR-Onlz3K28xfkPCk3_nBZs4D7CKnjgIaCgYKAf4SARESFQHGX2MiUlmJ3vFehn_auk4nsoEhQg0178


In [37]:
climate = ee.ImageCollection(exportPathRoot)
Map.clearMap()
Map.addLayer(climate.select([3,0,2]),{'canAreaChart':True,'areaChartParams':{'crs':crs,'transform':transform}'reducer':ee.Reducer.mean(),'min':[10,2,2],'max':[25,5,200]},'Climate Data')
print(climate.first().bandNames().getInfo())
Map.turnOnInspector()
Map.view(open_browser=True)

Adding layer: Climate Data
['prcp_mean', 'srad_mean', 'swe_mean', 'tmax_mean', 'tmin_mean', 'vp_mean']
Starting webmap
Using default refresh token for geeView
Local web server at: http://localhost:8001/geeView/ already serving.
cwd z:\Projects\06_LCMS_4_NFS\Scripts\landscape-change-data-explorer\src\setup
geeView URL: http://localhost:8001/geeView/?projectID=gtac-lamda&accessToken=ya29.a0AeDClZCoT_-G5-R4pj8tt0rIZBnlMXJ5ntHjCcRmSznacsolMIw0StvCqYG4qJ2f65UpVJ7lE4A3t0Rw5IM0fZPNywupnBKPZtNOe77uZQyPoyh7ZUCJySId5gb-R3ks_YqxTgXeQcwK6l1dbh5c_bUTA3MdXqHaOZ3EgXM4gqcaCgYKAQoSARESFQHGX2MikK3oOjd1XVCudi5qPcIi-A0178


In [34]:
Map.clearMap()
bandName = 'prcp_mean'
gil.changeDirDict[bandName] = 1
print(gil.changeDirDict)
run_params = {
    "maxSegments": 9,
    "spikeThreshold":5,
    "vertexCountOvershoot": 3,
    "preventOneYearRecovery": False,
    "recoveryThreshold": 5,
    "pvalThreshold": 0.05,
    "bestModelProportion": 0.25,
    "minObservationsNeeded": 6,
}
ltOutputs = cdl.simpleLANDTRENDR(
    climate,
    1985,
    2024,
    bandName,
    run_params,
    addToMap=True,
    multBy=10000,
)

rawLTForExport = ltOutputs[0]
print(rawLTForExport.getInfo())
Map.addLayer(rawLTForExport,{},'Raw LT')

Map.turnOnInspector()
Map.view(open_browser=True)
# lt = ee.Algorithms.TemporalSegmentation.LandTrendr(timeSeries, maxSegments, spikeThreshold, vertexCountOvershoot, preventOneYearRecovery, recoveryThreshold, pvalThreshold, bestModelProportion, minObservationsNeeded)

{'blue': 1, 'green': 1, 'red': 1, 'nir': -1, 'swir1': 1, 'swir2': 1, 'temp': 1, 'NDVI': -1, 'NBR': -1, 'NDMI': -1, 'NDSI': 1, 'brightness': 1, 'greenness': -1, 'wetness': -1, 'fourth': -1, 'fifth': 1, 'sixth': -1, 'ND_blue_green': -1, 'ND_blue_red': -1, 'ND_blue_nir': 1, 'ND_blue_swir1': -1, 'ND_blue_swir2': -1, 'ND_green_red': -1, 'ND_green_nir': 1, 'ND_green_swir1': -1, 'ND_green_swir2': -1, 'ND_red_swir1': -1, 'ND_red_swir2': -1, 'ND_nir_red': -1, 'ND_nir_swir1': -1, 'ND_nir_swir2': -1, 'ND_swir1_swir2': -1, 'R_swir1_nir': 1, 'R_red_swir1': -1, 'EVI': -1, 'SAVI': -1, 'IBI': 1, 'tcAngleBG': -1, 'tcAngleGW': -1, 'tcAngleBW': -1, 'tcDistBG': 1, 'tcDistGW': 1, 'tcDistBW': 1, 'NIRv': -1, 'NDCI': -1, 'NDGI': -1, 'prcp_mean': 1}
Converting LandTrendr from array output to Gain & Loss
Adding layer: Raw and Fitted Time Series
Adding layer: 1 prcp_mean Loss Year
Adding layer: 1 prcp_mean Loss Magnitude
Adding layer: 1 prcp_mean Loss Duration
Adding layer: 1 prcp_mean Gain Year
Adding layer: 1 